<a href="https://colab.research.google.com/github/ahsan-fayyaz/Artificial-Intelligence/blob/main/Assignment_3/Assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Imports section

In [1]:
# Imports section

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from IPython.display import Markdown as md

## Part 1. Loading the dataset

In [27]:
# Using pandas load the dataset (load remotely, not locally)
df = pd.read_csv("https://raw.githubusercontent.com/profmcnich/example_notebook/main/science_data_large.csv")
# Output the first 15 rows of the data
df.head(15)

,Temperature °C,Mols KCL,Size nm^3
0,469,647,6.244743e+05
1,403,694,5.779610e+05
2,302,975,6.196847e+05
3,779,916,1.460449e+06
4,901,18,4.325726e+04
5,545,637,7.124634e+05
6,660,519,7.006960e+05
7,143,869,2.718260e+05
8,89,461,8.919803e+04
9,294,776,4.770210e+05


#### Display summary of the table

In [4]:
# Display a summary of the table information (number of datapoints, etc.)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
Temperature °C    1000 non-null int64
Mols KCL          1000 non-null int64
Size nm^3         1000 non-null float64
dtypes: float64(1), int64(2)
memory usage: 23.5 KB


In [5]:
df.describe()

,Temperature °C,Mols KCL,Size nm^3
count,1000.000000,1000.000000,1.000000e+03
mean,500.500000,471.530000,5.086111e+05
std,288.819436,288.482872,4.474838e+05
min,1.000000,1.000000,1.611429e+01
25%,250.750000,226.750000,1.298267e+05
50%,500.500000,459.500000,3.827182e+05
75%,750.250000,710.250000,7.603211e+05
max,1000.000000,1000.000000,1.972127e+06


## Part 2. Splitting the dataset

In [6]:
print("Columns : ", df.columns)

Columns :  Index(['Temperature °C', 'Mols KCL', 'Size nm^3'], dtype='object')


In [28]:
# Take the pandas dataset and split it into our features (X) and label (y)

#features (X)
X = df[["Temperature °C", "Mols KCL"]]

#label (y)
y = df["Size nm^3"]

# Use sklearn to split the features and labels into a training/test set. (90% train, 10% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

## Part 3. Perform a Linear Regression

#### Make an object of the class LinearRegression followed by fit method which fits the regressor to the training data

In [9]:
# Use sklearn to train a model on the training set
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Call predict method on the actual dataset

In [10]:
# Create a sample datapoint and predict the output of that sample with the trained model
y_pred = model.predict(X_test)

#### Make a dataframe to compare the actual vs predicted

In [11]:
compare_actual_predicted = pd.DataFrame({"Actual" : y_test, 'Predicted' : y_pred})
compare_actual_predicted.head()

,Actual,Predicted
347,1.114779e+06,1.036778e+06
601,3.607683e+05,5.382207e+05
953,1.321831e+06,1.131998e+06
625,3.864304e+05,4.097998e+05
67,7.129771e+05,7.058629e+05


#### Use score() method which will use X_test to predict values above and compare with y_test values to tell the accuracy

In [12]:
# Report on the score for that model, in your own words (markdown, not code) explain what the score means
score = model.score(X_test, y_test)
print("Score = ", score)

Score =  0.869355108324308


#### What does the score here mean?
The scoring method for this linear regression is R squared. This metric quantifies the amount of variance in the target variable that is predicted from the feature variables. Here 86% means that the model has 86% accuracy predicting the size based upon temperature & mol variables

In [32]:
score_percentage = score*100
md("In this case the accuracy is %0.1f percent"%(score_percentage))

In this case the accuracy is 100.0 percent

#### Coefficients and Intercept

In [33]:
temp_coef = model.coef_[0]
mols_coef = model.coef_[1]
intercept = model.intercept_
print("Temperature Coefficient = ", temp_coef)
print("Mols Coefficient        = ", mols_coef)
print("Intercept = ", intercept)

Temperature Coefficient =  0.0
Mols Coefficient        =  12.000000023119355
Intercept =  4.344619810581207e-06


#### Equation

In [29]:
# Extract the coefficents and intercept from the model and write an equation for your h(x) using LaTeX

# **Note**: Since the variables are changing every time you run the notebook,
# I couldn't find a way to write variables in markdown. Therefore, I found this solution where I can write
# markdown in the code using this Markdown library. This lets me write variables inside this markdown


md("$h(x) = %i$ + %i*temp$ + %i*mol$"%(intercept, temp_coef, mols_coef))

$h(x) = -413912$ + 877*temp$ + 1022*mol$

## Part 4. Use Cross Validation

In [37]:
# Use the cross_val_score function to repeat your experiment across many shuffles of the data
# Report on their finding and their significance

#I will choose to use cv = 10 that means 10 folds of data
cv_results = cross_val_score(model, X_train, y_train,cv=10)
cv_results_mean, cv_results_std = cv_results.mean(), cv_results.std()
print("Cross Validation results : ",cv_results)


Cross Validation results :  [0.87625576 0.85948426 0.85768626 0.85909607 0.86119512 0.85669868
 0.85357203 0.8610199  0.84665108 0.85726163]


In [36]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results_mean, cv_results_std))

0.86 accuracy with a standard deviation of 0.01


#### Significance of using cross validation
When computing R squared on the test set, the R squared is dependent on the way you split up the data. The data points in the test set may have anomalies that which implies that the R-squared computed is not representative of the model's ability to generalize the unseen data. Cross validation here split the dataset into 10 folds/groups. This essentially took the first fold as a test set and fit the model on the remaining 9 folds. Then it predicted on the test set. This repeats for total 10 times. Eventually giving us an array of cross-validation scores. Therefore giving us a more accurate score value

## Part 5. Using Polynomial Regression

In [17]:
# Using the PolynomialFeatures library perform another regression on an augmented dataset of degree 2
# Here I used BayesianRidge() as my "another type of" regression

poly = PolynomialFeatures(degree=2)
X_test = poly.fit_transform(X_test)
X_train = poly.fit_transform(X_train)
model = BayesianRidge()
model.fit(X_train, y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [18]:
#Predict on the test data: y_pred
y_pred_new = model.predict(X_test)
compare_actual_predicted = pd.DataFrame({"Actual" : y_test, 'Predicted' : y_pred_new})
compare_actual_predicted.head()

,Actual,Predicted
347,1.114779e+06,1.114779e+06
601,3.607683e+05,3.607683e+05
953,1.321831e+06,1.321831e+06
625,3.864304e+05,3.864304e+05
67,7.129771e+05,7.129771e+05


In [19]:
score = model.score(X_test, y_test)
print("Score = ", score)

Score =  1.0


In [20]:
# Report on the metrics and output the resultant equation as you did in Part 3.
coef = model.coef_
print("Coefficients = ",coef)
print("Intercepts = ", model.intercept_)

Coefficients =  [ 0.00000000e+00  1.20000000e+01 -5.48898242e-08 -1.68292047e-11
  2.00000000e+00  2.85714286e-02]
Intercepts =  4.344619810581207e-06


#### Equation

In [31]:
md("$h(x) = %0.2f$ + %0.2fa$ + %0.2fb + %0.2fa^2 + %0.2fab$ + %0.2fb^2 "%(coef[0], coef[1],coef[2], coef[3], coef[4], coef[5]))

$h(x) = 0.00$ + 12.00a$ + -0.00b + -0.00a^2 + 2.00ab$ + 0.03b^2 

#### Report
I used BayesianRidge regression for this part. After using polynomial regression, the model's accuracy is now 1.0 i.e 100%. This implies that the model can predict sizes with an accuracy of 100% based upon the temperature & mol variables